# Exploratory Data Analysis (EDA) Tutorial — Google Colab

_Generated on 2025-10-28 13:30 UTC_

This notebook guides you through a practical, step-by-step EDA workflow in Google Colab.
It uses only standard scientific Python libraries and **matplotlib** for charts.


In [ ]:
# Install (if needed) and import dependencies
!pip -q install pandas numpy matplotlib scipy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import io, textwrap

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 120)
print(pd.__version__, np.__version__)


## Load Your Data
Choose one of the following approaches to load a dataset:
1) From a URL, 2) Upload from your computer, or 3) From Google Drive.

In [ ]:
import pandas as pd
from typing import Optional

# Option A: From a URL
URL: Optional[str] = ''  # e.g., 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'

df = None
if URL:
    df = pd.read_csv(URL)

# Option B: Upload from local machine
if df is None:
    try:
        from google.colab import files
        uploaded = files.upload()
        for fname in uploaded.keys():
            if fname.lower().endswith(('.csv', '.txt')):
                df = pd.read_csv(io.BytesIO(uploaded[fname]))
                break
            elif fname.lower().endswith(('.xlsx', '.xls')):
                df = pd.read_excel(io.BytesIO(uploaded[fname]))
                break
    except Exception as e:
        print('Upload skipped or not in Colab:', e)

# Option C: Load from Google Drive
if df is None:
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        # Example path: '/content/drive/MyDrive/path/to/data.csv'
        DRIVE_PATH = ''
        if DRIVE_PATH:
            if DRIVE_PATH.lower().endswith(('.csv', '.txt')):
                df = pd.read_csv(DRIVE_PATH)
            else:
                df = pd.read_excel(DRIVE_PATH)
    except Exception as e:
        print('Drive not mounted:', e)

assert df is not None, 'No dataset loaded. Please set URL, upload a file, or provide DRIVE_PATH.'
print('Loaded shape:', df.shape)
df.head()


## Quick Glance at the Data
Check the shape, columns, info, and first/last rows.

In [ ]:
print('Shape:', df.shape)
print('\nColumns:', list(df.columns))
print('\nInfo:')
df.info()
display(df.head())
display(df.tail())
display(df.sample(min(5, len(df))))


## Memory Usage and Dtypes
Identify potential optimizations for large datasets.

In [ ]:
mem = df.memory_usage(deep=True).sum()/1024**2
print(f'Total memory usage: {mem:.3f} MB')
display(df.dtypes)


## Missing Values Overview
Count and percent missing per column, and visualize the missingness pattern.

In [ ]:
na_count = df.isna().sum().sort_values(ascending=False)
na_pct = (df.isna().mean()*100).sort_values(ascending=False)
missing_summary = pd.DataFrame({'missing_count': na_count, 'missing_pct': na_pct})
display(missing_summary)

# Visualize missing matrix using matplotlib
plt.figure()
plt.imshow(df.isna(), aspect='auto', interpolation='nearest')
plt.title('Missing Data Pattern (rows x columns)')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()


## Duplicate Rows
Identify and optionally drop duplicates.

In [ ]:
dupe_count = df.duplicated().sum()
print('Duplicate rows:', dupe_count)
# Uncomment to drop
# df = df.drop_duplicates().reset_index(drop=True)


## Descriptive Statistics
Summary stats for numeric and categorical variables.

In [ ]:
display(df.describe(include=[np.number]).T)
display(df.describe(include=['object', 'category']).T)


## Cardinality and Top Categories
Check unique counts and most frequent values for categorical columns.

In [ ]:
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
summary = []
for c in cat_cols:
    vc = df[c].value_counts(dropna=False)
    summary.append({'column': c, 'unique': df[c].nunique(dropna=False), 'top_value': vc.index[0] if len(vc)>0 else None, 'top_count': int(vc.iloc[0]) if len(vc)>0 else 0})
display(pd.DataFrame(summary))


## Distributions — Histograms (Numeric)
Plot histograms for each numeric column.

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    plt.figure()
    plt.hist(df[col].dropna(), bins=30)
    plt.title(f'Histogram: {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()


## Distributions — Boxplots (Numeric)
Visualize spread and potential outliers.

In [ ]:
for col in num_cols:
    if df[col].dropna().shape[0] > 0:
        plt.figure()
        plt.boxplot(df[col].dropna(), vert=True)
        plt.title(f'Boxplot: {col}')
        plt.ylabel(col)
        plt.show()


## Categorical Frequencies — Top 15
Bar charts for the most frequent categories (per column).

In [ ]:
TOP_N = 15
for col in cat_cols:
    vc = df[col].astype('object').fillna('NA').value_counts().head(TOP_N)
    plt.figure()
    plt.bar(vc.index.astype(str), vc.values)
    plt.title(f'Top {TOP_N} Categories: {col}')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()


## Correlation Matrix (Numeric)
Compute and visualize correlations using matplotlib.

In [ ]:
if len(num_cols) >= 2:
    corr = df[num_cols].corr(numeric_only=True)
    plt.figure()
    plt.imshow(corr, cmap=None)
    plt.colorbar()
    plt.xticks(range(len(num_cols)), num_cols, rotation=90)
    plt.yticks(range(len(num_cols)), num_cols)
    plt.title('Correlation Matrix')
    plt.tight_layout()
    plt.show()
    display(corr)
else:
    print('Not enough numeric columns for correlation matrix.')


## Pairwise Scatter (Sampled)
For small datasets, this gives a quick view of linear relationships. For large datasets, we sample.

In [ ]:
from itertools import combinations
max_points = 2000
sample = df.sample(n=min(len(df), max_points), random_state=42) if len(df) > max_points else df.copy()
pairs = list(combinations(num_cols[:5], 2))  # limit to first 5 numeric cols for brevity
for x, y in pairs:
    plt.figure()
    plt.scatter(sample[x], sample[y], s=10)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(f'Scatter: {x} vs {y}')
    plt.tight_layout()
    plt.show()


## Outlier Detection (IQR Method)
Identify outliers per numeric column using the 1.5×IQR rule.

In [ ]:
def iqr_outliers(series: pd.Series):
    q1, q3 = np.percentile(series.dropna(), [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    mask = (series < lower) | (series > upper)
    return mask, lower, upper

outlier_summary = []
for col in num_cols:
    mask, lower, upper = iqr_outliers(df[col])
    outlier_summary.append({'column': col, 'outlier_count': int(mask.sum()), 'lower_bound': lower, 'upper_bound': upper})
display(pd.DataFrame(outlier_summary))


## Optional: Define a Target Column
If you have a target variable (e.g., for classification/regression), set it here for focused EDA.

In [ ]:
TARGET = ''  # e.g., 'Survived' for Titanic
if TARGET and TARGET in df.columns:
    print('Target dtype:', df[TARGET].dtype)
    if pd.api.types.is_numeric_dtype(df[TARGET]):
        plt.figure()
        plt.hist(df[TARGET].dropna(), bins=30)
        plt.title(f'Target Distribution: {TARGET}')
        plt.xlabel(TARGET)
        plt.ylabel('Frequency')
        plt.show()
    else:
        vc = df[TARGET].astype('object').fillna('NA').value_counts()
        plt.figure()
        plt.bar(vc.index.astype(str), vc.values)
        plt.title(f'Target Classes: {TARGET}')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('Count')
        plt.tight_layout()
        plt.show()
else:
    print('No TARGET set or not found in columns.')


## Numeric Columns vs Target (if defined)
Group summaries to compare distributions by target class.

In [ ]:
if TARGET and TARGET in df.columns and not pd.api.types.is_numeric_dtype(df[TARGET]):
    for col in num_cols:
        grp = df[[TARGET, col]].dropna().groupby(TARGET)[col].describe()
        print(f"\n{col} by {TARGET}")
        display(grp)
else:
    print('No categorical TARGET set for grouped numeric summaries.')


## Optional: Time Series Quick Look
If you have a date column, set it and view a simple trend.

In [ ]:
DATE_COL = ''  # e.g., 'date'
VALUE_COL = '' # a numeric column to aggregate
if DATE_COL and DATE_COL in df.columns and VALUE_COL and VALUE_COL in df.columns:
    tmp = df[[DATE_COL, VALUE_COL]].dropna().copy()
    tmp[DATE_COL] = pd.to_datetime(tmp[DATE_COL], errors='coerce')
    tmp = tmp.dropna(subset=[DATE_COL])
    tmp = tmp.set_index(DATE_COL).sort_index()
    ts = tmp[VALUE_COL].resample('D').mean()
    plt.figure()
    plt.plot(ts.index, ts.values)
    plt.title(f'Daily Mean of {VALUE_COL}')
    plt.xlabel('Date')
    plt.ylabel(VALUE_COL)
    plt.tight_layout()
    plt.show()
else:
    print('No DATE_COL/VALUE_COL set for time series section.')


## Optional: Simple Feature Engineering
Create a few example features (log transforms, simple ratios).

In [ ]:
ENGINEERED_FEATURES = []
for col in num_cols:
    if (df[col] > 0).all():
        df[f'log_{col}'] = np.log(df[col])
        ENGINEERED_FEATURES.append(f'log_{col}')
print('Engineered features:', ENGINEERED_FEATURES)
display(df.head())


## Export a Clean Snapshot
Save a working copy of the current dataframe to CSV.

In [ ]:
OUTPUT_CSV = 'eda_clean_snapshot.csv'
df.to_csv(OUTPUT_CSV, index=False)
print('Saved:', OUTPUT_CSV)


## Next Steps
- Explore relationships between variables more deeply.
- Consider statistical tests (t-tests, chi-square) appropriate to your data.
- Build baseline models (where relevant) for predictive tasks.
